In [1]:
import time
import numpy as np
import math

import torch
import torch.nn as nn
import torch.optim as optim

import DRLPDE_nn
import DRLPDE_functions.EvaluateWalkers
 
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300


In [2]:
### Taylor Green Parameters

mu = 1

# Physical Dimension
x_dim = 2
output_dim = 2

# Steady   or Unsteady
# Elliptic or Parabolic
is_unsteady = True
input_dim = x_dim + is_unsteady
time_range = [0,0.25]

################# Analytic Solution ######################

exists_analytic_sol = True
# If there is a true solution, provide contour levels
plot_levels = np.linspace(-1,1,100)

def true_solution(X):
    u = torch.stack( ( torch.cos(X[:,0])*torch.sin(X[:,1])*torch.exp(-2*mu*X[:,2]),
                       -torch.sin(X[:,0])*torch.cos(X[:,1])*torch.exp(-2*mu*X[:,2]) ), dim=1)
    return u


################# PDE Coefficients ########################

# PDE type:
pde_type = 'NavierStokes'

# Diffusion coefficient
# mu = 1

# Forcing term
def forcing(X):
    f = torch.zeros( (X.size(0), output_dim), device=X.device)
    return f

################# Boundary and Initial Conditions ###########
# Use pytorch expressions to make boundary and initial conditions 
#
# To make different boundary conditions for each boundary
#     ensure the correct bdry_con is called when defining the boundaries

def bdry_con(X):
    u = torch.zeros( (X.size(0), output_dim), device=X.device)
    return u

def init_con(X):
    u = torch.stack( ( torch.cos(X[:,0])*torch.sin(X[:,1]),
                       -torch.sin(X[:,0])*torch.cos(X[:,1]) ), dim=1)
    return u

#################  Make the domain  #######################

boundingbox = [ [-math.pi, math.pi], [-math.pi,math.pi], ]

periodic1 = { 'variable':'x', 
              'base':-math.pi,
              'top':math.pi }

periodic2 = { 'variable':'y', 
              'base':-math.pi,
              'top':math.pi }

list_of_dirichlet_boundaries = []
list_of_periodic_boundaries =[periodic1, periodic2]


In [3]:
# Time step
dt = 1e-2
# exit tolerance
tol = 1e-6

# Number of walkers

numpts = 2**13

num_ghost = 512
num_batch = 2**10

num_fixed = 128

# 
use_true_vel = False

# Update walkers
# Options: 
#    move -- moves walkers to one of their new locations
#    remake -- remake walkers at each training step
#    fixed -- keeps walkers fixed
update_walkers = 'move'
update_walkers_every = 1
############## Training Parameters #######################

# Training epochs
num_epoch = 1000

update_print_every = 500

# Neural Network Architecture
nn_depth = 60
nn_width = 4

# Weighting of losses
lambda_bell = 1e2
lambda_fix = 1e0

# Learning rate
learning_rate = 1e-3
adam_beta = (0.9,0.999)
weight_decay = 1e-4


In [4]:
class FeedForwardNN(nn.Module):
    
    ### Feed forward neural network
    
    def __init__(self, depth, width, x_dim, is_unsteady, output_dim, **nn_param):
        super(FeedForwardNN, self).__init__()
        
        self.input_dim = x_dim + is_unsteady
        
        modules = []
        modules.append(nn.Linear(self.input_dim, depth))
        for i in range(width - 1):
            modules.append(nn.Linear(depth, depth))
            modules.append(nn.ELU())
        modules.append(nn.Linear(depth, 1))
        
        self.sequential_model = nn.Sequential(*modules)
        
    def forward(self, x):
        a = self.sequential_model(x)
        
        return a

class PotentialNN(nn.Module):
    
    def __init__(self, depth, width, x_dim, is_unsteady, output_dim, **nn_param):
        super(PotentialNN, self).__init__()
        
        self.x_dim = x_dim
        self.input_dim = x_dim + is_unsteady
        
        modules = []
        modules.append(nn.Linear(self.input_dim, depth))
        for i in range(width - 1):
            modules.append(nn.Linear(depth, depth))
            modules.append(nn.ELU())
        modules.append(nn.Linear(depth, 1))
        
        self.sequential_model = nn.Sequential(*modules)
        
    def forward(self, x):
        p = self.sequential_model(x)
        
        a = torch.autograd.grad(p, x, grad_outputs = torch.ones_like(p), create_graph = True, 
                                        retain_graph = True, only_inputs = True)[0]

        return a[:,:self.x_dim]

    def evaluate_potential(self,x):
        p = self.sequential_model(x)

        return p

def eval_gradient(X, model):
    #X.requires_grad = True
    a = model(X)

    grad_model = torch.autograd.grad(a, X, grad_outputs = torch.ones_like(a), create_graph = True, 
                                       retain_graph = True, only_inputs = True)[0]

    return grad_model[:,:x_dim]

In [5]:
class DefineDomain:
    ### This class defines the domain using the parameters provided
    ### It sets up the boundingbox and each boundary

    def __init__(self, is_unsteady, boundingbox, 
                list_of_dirichlet_boundaries,
                list_of_periodic_boundaries):
        
        self.boundingbox = boundingbox
        self.is_unsteady = is_unsteady

        self.num_of_boundaries = len(list_of_dirichlet_boundaries)
        
        # Unpack dirichlet boundary descriptions
        self.boundaries = []
        for specs in list_of_dirichlet_boundaries:
            ### 2D boundaries
            if specs['type'] == 'line':
                self.boundaries.append( bdry_line( point = specs['point'], 
                                                   normal = specs['normal'],
                                                   endpoints = specs['endpoints'],
                                                   boundary_condition = specs['boundary_condition'] ))
        # Unpack any periodic boundaries
        self.periodic_boundaries = []
        for specs in list_of_periodic_boundaries:
            self.periodic_boundaries.append( bdry_periodic( variable = specs['variable'],
                                                            base = specs['base'],
                                                            top = specs['top']  ))
class bdry_periodic:

    def __init__(self, variable, base, top):
        
        if variable == 'x':
            self.index = 0
        if variable == 'y':
            self.index = 1
        if variable == 'z':
            self.index = 2

        self.base = base
        self.top = top

def generate_interior_points( numpts, boundingbox, boundaries):
    ### Generate points inside the domain
    
    ### Uniform Grid
    #x1 = (boundingbox[0][1] - boundingbox[0][0])*torch.linspace( dt, 1-dt, numpts_x ) + boundingbox[0][0]
    #x2 = (boundingbox[1][1] - boundingbox[1][0])*torch.linspace( dt, 1-dt, numpts_y ) + boundingbox[1][0]
    #t_var = (time_range[1] - time_range[0])*torch.linspace( dt, 1-dt, numpts_t) + time_range[0]
    #X = torch.cartesian_prod( x1, x2, t_var)

    ### Randomly
    x1 = (boundingbox[0][1] - boundingbox[0][0])*torch.rand(numpts) + boundingbox[0][0]
    x2 = (boundingbox[1][1] - boundingbox[1][0])*torch.rand(numpts) + boundingbox[1][0]
    t_var = (time_range[1] - time_range[0])*torch.sqrt( torch.rand(numpts) ) + time_range[0]
    X = torch.stack([x1, x2, t_var], dim=1)

    return X

def periodic_condition(Xnew, periodic_boundaries):
    for bdry in periodic_boundaries:
        below_base = Xnew[:,bdry.index] < bdry.base
        above_top = Xnew[:,bdry.index] > bdry.top

        if torch.sum(below_base) > 0:
            Xnew[below_base, bdry.index] = Xnew[below_base, bdry.index] + (bdry.top - bdry.base)
        if torch.sum(above_top) > 0:
            Xnew[above_top,bdry.index] = Xnew[above_top, bdry.index] - (bdry.top - bdry.base)
    return Xnew

def find_time_exit(Xold, Xnew, tol):
    ### Bisection algorithm to find the time exit up to a tolerance
    
    Xmid = (Xnew + Xold)/2

    # above tolerance = inside
    # below tolerance = outside
    above_tol = Xmid[:,-1] > tol
    below_tol = Xmid[:,-1] < -tol

    if torch.sum(above_tol + below_tol) > 0:
        Xnew[below_tol,:] = Xmid[below_tol,:]
        Xold[above_tol,:] = Xmid[above_tol,:]
        
        Xmid[above_tol + below_tol,:] = find_time_exit(Xold[above_tol + below_tol,:], Xnew[above_tol + below_tol,:], tol)

    return Xmid

def make_target(X):
    
    numpts = X.size(0)

    ### Evaluate at locations
    if use_true_vel:
        Uold = true_solution(X)
    else:
        Uold = model_velocity(X)

    ### Move Walkers
    Zt = np.sqrt(dt)*torch.randn((numpts*num_ghost, x_dim), device=dev)

    Xnew = X.detach().repeat(num_ghost,1)  + torch.cat( (-dt*Uold.detach().repeat(num_ghost,1) + np.sqrt(2*mu)*Zt, 
                                                            -dt*torch.ones((numpts*num_ghost,1), device=dev)), dim=1)


    ### Initial condition
    hit_initial = Xnew[:,-1] < 0
    Xnew[hit_initial,:] = find_time_exit(X.repeat(num_ghost,1)[hit_initial,:], Xnew[hit_initial,:], tol)

    ### Calculate periodic boundaries
    Xnew = periodic_condition(Xnew, Domain.periodic_boundaries)

    ### Evaluate at locations
    if use_true_vel:
        Unew = true_solution(Xnew).reshape(num_ghost, numpts, output_dim).mean(0)
    else:
        Unew = model_velocity(Xnew).reshape(num_ghost, numpts, output_dim).mean(0)
        
    ### Make target
    target = (Unew - Uold).detach()/dt

    return Xnew, target

class Walker_Data(torch.utils.data.Dataset):
    
    def __init__(self, boundingbox, boundaries):

        self.Xold = generate_interior_points(numpts, boundingbox, boundaries).requires_grad_(True)

    def __len__(self):
        ### How many data points are there?
        return numpts
    
    def __getitem__(self, index):
        ### Gets one sample of data
        ### 
        return self.Xold[index,:], index


In [6]:
nn_param = {'depth': nn_depth,
            'width': nn_width,
            'x_dim': x_dim,
            'is_unsteady': is_unsteady ,
            'output_dim': output_dim
            }

vel_nn_param = {'depth': 60,
            'width': 4,
            'x_dim': 2,
            'is_unsteady': True ,
            'output_dim': 2
            }

eval_model_param={'dt': dt,
                  'forcing': forcing}

### Make boundaries defining the domain
Domain = DefineDomain(is_unsteady, boundingbox, list_of_dirichlet_boundaries, list_of_periodic_boundaries)

Velocity_NN = DRLPDE_nn.IncompressibleNN
model_velocity = Velocity_NN(**vel_nn_param)
model_velocity.load_state_dict(torch.load("savedmodels/TaylorGreen.pt"))
model_velocity.to(dev).eval()

### Initialize the Model
MyNeuralNetwork = FeedForwardNN
model_pressure = MyNeuralNetwork(**nn_param).to(dev)

mseloss = nn.MSELoss(reduction = 'mean')
optimizer = optim.Adam(model_pressure.parameters(), 
                        lr=learning_rate, 
                        betas=adam_beta, 
                        weight_decay=weight_decay)

### Create Walkers and Boundary points and Organize into DataLoader
RWalkers = Walker_Data(boundingbox, Domain.boundaries)
RWalkers_batch = torch.utils.data.DataLoader(RWalkers, batch_size=num_batch, shuffle=True)

if update_walkers == 'move':
    move_RWalkers = torch.zeros_like(RWalkers.Xold)


### Fixed pressure
x_fix = torch.stack( [torch.zeros((num_fixed)), torch.zeros((num_fixed)), torch.linspace(time_range[0],time_range[1], num_fixed)], dim=1).to(dev)
p_fix  = torch.zeros((num_fixed,1), device=dev)



In [7]:
start_time = time.time()

for step in range(num_epoch):

    # Try fixing one point to have 0 pressure
    loss = lambda_fix*mseloss( model_pressure(x_fix), p_fix)
    loss.backward()

    # Random Walkers - do in batches
    for Xold, index in RWalkers_batch:

        # Send to GPU and set requires grad flag
        Xold = Xold.to(dev).requires_grad_(True)

        ### Make the target points
        Xnew, target = make_target(Xold)

        target = target.detach()

        # No backprop wrt Xnew
        #grad_pressure = model_pressure(Xold)
        #grad_pressure = eval_gradient(Xold, model_pressure)
        grad_pressure = (eval_gradient(Xold, model_pressure) + eval_gradient(Xnew, model_pressure).reshape(num_ghost, num_batch, output_dim).mean(0).detach() )/2

        # Calculate loss
        loss = lambda_bell*mseloss(grad_pressure, target)
        loss.backward()

        if (step+1) % update_walkers_every == 0:
            move_RWalkers[index,:] = Xnew[:num_batch].detach().cpu()

    optimizer.step()
    optimizer.zero_grad()

    if (step+1) % update_walkers_every == 0:
        move_RWalkers = periodic_condition(move_RWalkers, Domain.periodic_boundaries)
        outside = move_RWalkers[:,-1] < 0
        move_RWalkers[outside,:] = generate_interior_points( torch.sum(outside), boundingbox, Domain.boundaries)

        RWalkers.Xold = move_RWalkers

    if step == 0:
        print('No errors in first epoch: Training will continue')
    if (step+1) % update_print_every == 0:
        current_time = time.time()
        np.set_printoptions(precision=2)
        print('step = {0}/{1}, {2:2.3f} s/step, time-to-go:{3:2.0f}s'.format(
                step+1, num_epoch, (current_time - start_time) / (step + 1), 
            (current_time - start_time) / (step + 1) * (num_epoch - step - 1)))
        print('loss bell = {:.4f}'.format(lambda_bell*mseloss(grad_pressure, target).detach().cpu().numpy()))
        print('loss fix = {:.4f}'.format(lambda_fix*mseloss( model_pressure(x_fix), p_fix).detach().cpu().numpy()))
    

No errors in first epoch: Training will continue
step = 500/1000, 0.467 s/step, time-to-go:233s
loss bell = 18.3064
loss fix = 0.0005
step = 1000/1000, 0.466 s/step, time-to-go: 0s
loss bell = 17.1216
loss fix = 0.0000


In [8]:
torch.save(model_pressure.state_dict(), "savedmodels/" + 'pressureTaylorGreen'+ ".pt")